# Task 2

We want to get into a company e.g. AWS and want to know the minimum number of steps to come into contact with someone who works/has worked in the company


In [19]:
import sys
import os

src_path = '..'
sys.path.append(src_path)

In [20]:
from src.utils import neo4j_utils

neo_creds = {
    'host': 'localhost',
    'port': '7687',
    'user': 'neo4j',
    'password': 'neo4j1'
}
neoCon = neo4j_utils.neo_connector(creds=neo_creds)

In [21]:
# Application of shortest path
name = 'Abdullah Ashfaq'
public_identifier = 'aashfaq11'
educational_institution = '.*University of California.*San Diego.*'
groups = ['Data Science for Finance and Economics']
target_company = 'Google'

# Starting points are my groups and universities
starting_groups = []
starting_uni = ['.*University of California.*San Diego.*']



In [22]:
# Inserting myself in the network
insert_query = f'''
Merge (a:People {{full_name: "{name}", public_identifier: "{public_identifier}"}})
'''
neoCon.runQuery(insert_query)

[]

In [23]:
# Creating initial my connections
insert_query = f'''
Match (a:People {{full_name: "{name}", public_identifier: "{public_identifier}"}}), (u:Edu_Institution), (g:Group)
where u.name =~ '(?i){educational_institution}' and g.name = '{groups[0]}'
merge (a)-[:STUDIED_IN]->(u)
merge (g)<-[:Is_In_Group]-(a)
'''
print(f"Running query: \n {insert_query}")
neoCon.runQuery(insert_query)

Running query: 
 
Match (a:People {full_name: "Abdullah Ashfaq", public_identifier: "aashfaq11"}), (u:Edu_Institution), (g:Group)
where u.name =~ '(?i).*University of California.*San Diego.*' and g.name = 'Data Science for Finance and Economics'
merge (a)-[:STUDIED_IN]->(u)
merge (g)<-[:Is_In_Group]-(a)



[]

In [24]:
# We want a path that doesn't pass through a Company node
# Because Paths involving company nodes and relations (e.g. works in domain) made less sense e.g. Me--(CompanyA)--(domain)--(Google)
query = f'''
MATCH (end:People)-[:EMPLOYEE_IN]->(c:Company)
WHERE c.name =~ '{target_company}'
WITH collect(end) as endPersons
Match (start:People {{full_name: "{name}", public_identifier: "{public_identifier}"}})
CALL apoc.path.subgraphNodes(start, {{relationshipFilter:'', endNodes:endPersons, limit:3, labelFilter:'-Company'}}) YIELD node
RETURN node
'''
print(f"Running query: \n {query}")
values = neoCon.runQuery(query)

Running query: 
 
MATCH (end:People)-[:EMPLOYEE_IN]->(c:Company)
WHERE c.name =~ 'Google'
WITH collect(end) as endPersons
Match (start:People {full_name: "Abdullah Ashfaq", public_identifier: "aashfaq11"})
CALL apoc.path.subgraphNodes(start, {relationshipFilter:'', endNodes:endPersons, limit:3, labelFilter:'-Company'}) YIELD node
RETURN node



In [14]:
# Our top 3 contacts
pub_identif = [x['node']['public_identifier'] for x in values]
pub_identif

['aakratikinger', 'aallaman', 'aaron-abaribe']

In [18]:
all_paths = []
for p in pub_identif:
    query = f'''
    Match p=shortestPath((a:People {{full_name: "{name}", public_identifier: "{public_identifier}"}})-[*]-(p1:People {{public_identifier:'{p}'}}))
    return p;
    '''
    print(f"Running query: \n {query}")
    values = neoCon.runQuery(query)
    all_paths.append(values)

all_paths

Running query: 
 
    Match p=shortestPath((a:People {full_name: "Abdullah Ashfaq", public_identifier: "aashfaq11"})-[*]-(p1:People {public_identifier:'aakratikinger'}))
    return p;
    
Running query: 
 
    Match p=shortestPath((a:People {full_name: "Abdullah Ashfaq", public_identifier: "aashfaq11"})-[*]-(p1:People {public_identifier:'aallaman'}))
    return p;
    
Running query: 
 
    Match p=shortestPath((a:People {full_name: "Abdullah Ashfaq", public_identifier: "aashfaq11"})-[*]-(p1:People {public_identifier:'aaron-abaribe'}))
    return p;
    


[[{'p': [{'public_identifier': 'aashfaq11', 'full_name': 'Abdullah Ashfaq'},
    'Is_In_Group',
    {'name': 'Data Science for Finance and Economics',
     'url': 'https://www.linkedin.com/groups/6781274'},
    'Is_In_Group',
    {'public_identifier': 'aamashharoon', 'full_name': 'Aamash Haroon'},
    'Is_In_Group',
    {'name': 'CFA Program Candidates',
     'url': 'https://www.linkedin.com/groups/38728'},
    'Is_In_Group',
    {'public_identifier': 'aakratikinger', 'full_name': 'Aakrati Kinger'}]}],
 [{'p': [{'public_identifier': 'aashfaq11', 'full_name': 'Abdullah Ashfaq'},
    'STUDIED_IN',
    {'name': 'University of California, San Diego',
     'linkedin_profile': 'https://www.linkedin.com/school/ucsandiego/'},
    'STUDIED_IN',
    {'public_identifier': 'aamirssyed', 'full_name': 'aamir syed'},
    'Is_In_Group',
    {'name': 'Social Media Marketing Group by Connect365.io',
     'url': 'https://www.linkedin.com/groups/66325'},
    'Is_In_Group',
    {'public_identifier': 'aalla